<a href="https://colab.research.google.com/github/Jaejuna/SpeedWagon/blob/main/KoBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/SKT-AI/KoBART#how-to-install

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers
!pip install Rouge
!pip install boto3
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
import transformers
import tensorflow

In [6]:
!git clone https://github.com/SKT-AI/KoBART.git

Cloning into 'KoBART'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 331 (delta 13), reused 14 (delta 13), pack-reused 303
Receiving objects: 100% (331/331), 8.24 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
%pwd

'/content'

In [7]:
%cd KoBART

/content/KoBART


In [ ]:
torch.__version__

'2.0.1+cu118'

In [ ]:
pip uninstall torch

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.0.1+cu118.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-2.0.1+cu118


In [ ]:
pip install torch==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.7.1


In [8]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.2/814.2 kB 11.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.7.1


In [9]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

/content/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [10]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train_new.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid_new.csv')

In [23]:
train_df.shape

(27993, 3)

In [24]:
valid_df.shape

(3496, 3)

In [25]:
train_df = train_df.iloc[:10000,:]

In [26]:
train_df.shape

(10000, 3)

In [11]:
import transformers
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import torch
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from datasets import Dataset
from tqdm import tqdm
import glob
import json
import os

In [27]:
# DataFrame을 Dataset으로 전환
import datasets
from datasets import Dataset
train_data = Dataset.from_pandas(train_df) 
valid_data = Dataset.from_pandas(valid_df)
test_samples = Dataset.from_pandas(valid_df)

print(train_data)
print(valid_data)
print(test_samples)

Dataset({
    features: ['Text', 'Summary', 'Topic'],
    num_rows: 10000
})
Dataset({
    features: ['Text', 'Summary', 'Topic'],
    num_rows: 3496
})
Dataset({
    features: ['Text', 'Summary', 'Topic'],
    num_rows: 3496
})


In [14]:
from transformers import BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

model_checkpoints = '/content/drive/MyDrive/BOAZ_miniProject2'
kobart_tokenizer = get_kobart_tokenizer(model_checkpoints)
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model(model_checkpoints))

/content/drive/MyDrive/BOAZ_miniProject2/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


using cached model. /content/KoBART/.cache/kobart_base_cased_ff4bda5738.zip


In [50]:
# 예시 텍스트
text = """
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""
text = text.replace('\n', ' ')

raw_input_ids = kobart_tokenizer.encode(text)
input_ids = [kobart_tokenizer.bos_token_id] + raw_input_ids + [kobart_tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'확진자가 급증하고 있다.'

In [59]:
valid_df.iloc[3495,0]

'아 공연늦겠다 집들려서 짐놓고 빨리나왓는데 ㅠ 헉 빨리가야지뭐ㅠㅠ 나는 준비중 그렇구만 화장중이예용 #@이름#이 오늘 열심히꾸미나 지금늦어서 거의다와가지고 택시탔어'

In [58]:
raw_input_ids = kobart_tokenizer.encode(valid_df.iloc[3495,0])
input_ids = [kobart_tokenizer.bos_token_id] + raw_input_ids + [kobart_tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'공연중려서 빨리나왔는데 집들려서 짐놓고 빨리나왔는데 화장중이 되었다.'

In [28]:
# 훈련 데이터 로드 및 전처리
input_ids = []
attention_masks = []

for data in train_data:
    encoded_data = kobart_tokenizer.encode_plus(
        data["Text"],
        max_length=300,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    input_ids.append(encoded_data["input_ids"])
    attention_masks.append(encoded_data["attention_mask"])

In [29]:
output_ids = []

for data in train_data:
    encoded_data = kobart_tokenizer.encode_plus(
        data["Summary"],
        max_length=300,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    output_ids.append(encoded_data["input_ids"])
    attention_masks.append(encoded_data["attention_mask"])

In [30]:
# 훈련 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

In [31]:
len(train_df)

10000

In [ ]:
# 훈련
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i in range(len(input_ids)):
        inputs = {
            "input_ids": input_ids[i].to(device),
            "attention_mask": attention_masks[i].to(device),
            "labels": output_ids[i].to(device)
        }
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i % 1000 == 0):
          print(f"{i}번째 complete")

    average_loss = total_loss / len(input_ids)
    print(f"Epoch {epoch+1} / {num_epochs} - Average Loss: {average_loss}")

In [60]:
# 모델 저장
import joblib 
joblib.dump(model, '/content/drive/MyDrive/BOAZ_miniProject2/kobart1.pkl')

['/content/drive/MyDrive/BOAZ_miniProject2/kobart1.pkl']

In [ ]:
# 모델 불러오기
model = joblib.load("/content/drive/MyDrive/BOAZ_miniProject2/kobart1.pkl")

In [35]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction

# 요약 및 ROUGE 점수 비교
references = []
summaries = []

for data in valid_data:
    # 입력 문장 및 요약 대상 문장 가져오기
    input_text = data["Text"]
    target_summary = data["Summary"]

    # KoBART에 입력 문장 토크나이징
    input_ids = kobart_tokenizer.encode(input_text, truncation=True, max_length=300, padding="max_length", return_tensors="pt")

    # KoBART를 사용하여 요약 생성
    summary_ids = model.generate(input_ids=input_ids.to(model.device), max_length=150, num_beams=4, early_stopping=True)
    summary = kobart_tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # 결과 저장
    references.append([target_summary])
    summaries.append(summary)

# ROUGE 점수 계산
rouge_scores = corpus_bleu(references, summaries, smoothing_function=SmoothingFunction().method4)
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_scores[0] * 100:.2f}")
print(f"ROUGE-2: {rouge_scores[1] * 100:.2f}")
print(f"ROUGE-L: {rouge_scores[2] * 100:.2f}")

KeyboardInterrupt: ignored

In [36]:
summaries

['닉장갓다온 날 회사 안 가고 집에 못 갔다.',
 '판교에서 또 막혔다.',
 '버스 기다리구이쪄 버스가너무 늦게 왔다.',
 '아양교까지 막힌다.',
 '이따 그친다고 한다.',
 '유엘 유엘에 대해 이야기하고 있다.',
 '이번 추율 돌아다니고할 때 별로 안 추운대 한파면 가기 싫다.',
 '3번째틀렷어레이(틀렷어레이)라고 한다.',
 '카드 마니(마니) 썼는데 돈아껴야 한다.',
 '카페 택배는 다 받았다.',
 '주말에도 심하면 산책도 못하겠다.',
 '버스가 8시10분꺼가 있다.',
 '엑소 나왔다.',
 '롱패딩이 아니더라도 롱패딩을 입으면 롱패딩이 된다.',
 '만년필은 잉크를 넣어야 한다.',
 '궐마에서 웰킵스마스크 1400원에 팔았다.',
 '설거지하는 게 또 고장나서 물이 사방에 튀고 있다.',
 '커버 얼룩이 많다고 미리 이야기하고 있다.',
 '아빠가 공고 전에 돈을 넣어야 한다.',
 '아이스크림 먹고 책을 읽어주고 있다.',
 '재난지원금 털기가 아직 남았는데 식비로 쓰기로 했다.',
 '파랑색 귀여운 버스를 다시 올라왔다.',
 '구천구백원은 내 팬티값이다.']

In [49]:
references

[['멘무샤가 없어져서 슬프고 롯데몰에 다양한 음식점이 생겨서 가보기로 했다.'],
 ['판교에서 가는 중인데 7시 30분에 도착하기는 힘들 것 같다.'],
 ['도착하기 4번째 전 정거장에서 말해주기로 한다.'],
 ['둘 다 차가 많이 막혀 먼저 도착한 사람이 미리 시켜놓기로 한다.'],
 ['비가 거의 그친 것 같고 추워진다고 하니 따듯하게 입고 다니라고 한다.'],
 ['유엘도 나이 제한이 변경됐는지 원래 65세였는데 안 돼서 저축 같은 걸로 설계를 해 보라고 한다.'],
 ['한파가 온다더니 이번 겨울은 별로 안 춥다.'],
 ['공인 인증서 비밀번호가 3번째 틀려서 차분히 눌러봐야겠다.'],
 ['이번 달 카드값을 너무 많이 써서 마사지를 못 받으러 간다고 하고 어디에 그렇게 썼는지 확인한다.'],
 ['카페로 온 택배를 다 받았는지 묻고 5시 퇴근 직전에 받았다고 대답한다.'],
 ['주말에 미세먼지가 심하지 않아서 산책할 수 있기를 바라고 있다.'],
 ['8시 10분 버스가 있는데 안산으로 가는 거라고 하니 안산에 내리면 데려다준다며 고민을 해봐야겠다고 한다.'],
 ['엑소 카이의 집이 나 혼자 산다(나혼산)에 나왔는데 인위적이고 인테리어에 돈을 많이 쓴 느낌이다.'],
 ['밖이 추워서 무조건 패딩을 입고 만나자고 한다.'],
 ['만년필과 뱀이 붙은 디자인의 볼펜에 대해 이야기한다.'],
 ['오늘 마트에서 월킵스 마스크가 1400원에 팔았다고 하며 오프라인에 점점 풀리는 거 같다고 한다.'],
 ['하나를 고치면 다른 하나가 고장 나는데 너무 싼 것을 썼는지 설거지하는 데 물이 나오는 주방 헤드가 고장이 났다.'],
 ['얼룩이 많은 매트리스 판매에 대해 이야기한다.'],
 ['엄마는 2순위라서 월요일에 돈을 넣어야 한다.'],
 ['아이가 낮잠을 잘 자서 늦게 재운다고 이야기한다.'],
 ['남자 친구의 재난지원금을 털기 위해 인천에 가는 중이다.'],
 ['파랑색 장미모양 무늬가 있는 물건이 귀엽다.'],
 ['친구들에게 39000원

In [42]:
rouge_scores = corpus_bleu(references, summaries, smoothing_function=SmoothingFunction().method4)
rouge_scores

0.10047413670532138

In [61]:
input_text = train_df.iloc[0,0]

# 문장을 토큰화하고 입력 인코딩
inputs = kobart_tokenizer(input_text, truncation=True, padding="longest", return_tensors="pt")

# 모델에 입력 전달하여 요약 생성
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = kobart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 출력
print("Input:", input_text)
print("Summary:", summary)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input: 퇴근길에 365코너 들러서 이체했음! OTP는 조만간 은행가야겠다 otp가..뭐야.. 난..몰라...오티피..처음들어봐 아 몰라? 그..인터넷계좌? 그런거잘몰라 인터넷뱅킹할때 쓰는 1회용 비밀번호 보여주는 조그마한 거 아아아 그..그거.. 매번 거래시 1회용 비번보여주ㅕ 인터넷 뱅킹이 편해져요! 공인인증서그런거비슷한거? 투명한카드? 아 내껀 카드형아님 옛날엔 보안카드 였다 공인인증서때 쓰는 카드 아그래..? 인터넷뱅킹 공인인증서+보안카드가 엿같아보여서 안쓴다.. 근데 그게 OTP로 바뀌었다 이잉..할미는 요즘 어쩌구 그렁거몰라
Summary: 인터넷뱅킹을 할 때 쓰는 1회용 비밀번호를 보여주는 것 같다.
